In [ ]:
#installer
%pip install tensorflow

In [ ]:
import numpy as np
import pandas as pd
from pathlib import Path
import os.path
import matplotlib.pyplot as plt
import tensorflow as tf
import warnings
warnings.filterwarnings("ignore")

# Import
data_path = 'input'
train_dir = Path(data_path + '/train')
train_filepaths = list(train_dir.glob(r'**/*.jpg'))

test_dir = Path(data_path + '/test')
test_filepaths = list(test_dir.glob(r'**/*.jpg'))

val_dir = Path(data_path + 'validation')
val_filepaths = list(test_dir.glob(r'**/*.jpg'))

def process_imgs(filepath):
    labels = [Path(filepath[i]).parent.name for i in range(len(filepath))]
    filepath = pd.Series(filepath, name='Filepath').astype(str)
    labels = pd.Series(labels, name='Label')
    
    dt = pd.concat([filepath, labels], axis=1)

    # Shuffle 
    dt = dt.sample(frac=1).reset_index(drop = True)
    
    return dt

train_dt = process_imgs(train_filepaths)
test_dt = process_imgs(test_filepaths)
val_dt = process_imgs(val_filepaths)

C:\Users\balog\AppData\Local\Temp\ipykernel_67884\2831523222.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [ ]:
#Check imports
print('-- Training set --\n')
print(f'Number of pictures: {train_dt.shape[0]}\n')
print(f'Number of different labels: {len(train_dt.Label.unique())}\n')
print(f'Labels: {train_dt.Label.unique()}')

train_dt.head(5)